# FIR filter design via the impulse response

FIR filter design with Hanning Window

### Preamble
Start by importing the Python libraries that we will require

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

And define a function that will return true if running in a Jupyter Notebook

In [ ]:
def is_jupyter():
    """Return true if running in a Jupyter Notebook"""
    try:
        if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
            return True
        else:
            return False
    except: 
        return False

### Define plotting functions
First define a function to plot the impulse response.

In [ ]:
def plot_stem(x, y, M, name):
    """
       Plot FIR impulse response.
       INPUT:
           x (array-like): The x-positions of the stems.
           y (array-like): The y-values of the stem heads.
           M        (int): The upper limit of x axis.
           name  (string): The name used to save figure.
    """
    # Create the plot figure
    plt.figure(figsize = (12, 6))
    
    # Enlarge figure label and axis size
    plt.rcParams.update({'font.size': 16})
    
    # stem plot  
    (markerLines, stemLines, baseLines) = plt.stem(x, y, use_line_collection = True)
    plt.setp(baseLines, color = 'black', linewidth=0.8) 
    plt.setp(stemLines, linewidth=1.5) 
    markerLines.set_markersize(2)
    
    # Tidy up the plot to control axes sizes and labels
    plt.xlabel('n')
    plt.ylabel('h(n)')
    plt.xlim([0, M-1])
    
    # Save figure in python or ipython system
    if not is_jupyter():
        plt.tight_layout()
        plt.savefig(name)

Now define another function to plot the frequency response

In [ ]:
def plot_freq(x, y, name):
    """
       Plot the magnitude of normalised freuqnecy
       INPUT:
           x (array-like): The horizontal coordinates of the data points.
           y (array-like): The vertical coordinates of the data points.
           name  (string): The name used to save figure.
    """
    # Create the plot figure
    plt.figure(figsize = (12, 6))
    
    # Enlarge figure label and axis size
    plt.rcParams.update({'font.size': 16})
    
    # Plot the frequency
    plt.plot(x, y)
    
    # Tidy up the plot to control axes sizes and labels
    plt.xlabel('Normalised frequency')
    plt.ylabel('Magnitude (dB)')
    plt.xlim([0, 0.1])
    plt.ylim([-80, 10])
    plt.xticks(np.linspace(0, 0.1, 11))
    
    # Save figure in python or ipython system
    if not is_jupyter():
        plt.tight_layout()
        plt.savefig(name)

### Define the impulse response
We define the impulse response as
$$\frac{1}{\pi n}\sin\left(\frac{\pi n}{10}\right)$$
This is the ideal impulse response windowed by a rectangular window function, and valid for $$-\frac{M}{2}<n<\frac{M}{2}$$

In [ ]:
# First define the sample points
M = 311
mid_point = (M-1) / 2
n = np.arange(0, M)

distance = n - mid_point

# Avoid division by zero error being flagged
old_settings=np.seterr(divide='ignore', invalid='ignore')
# Calculate the ideal impulse response
h_ideal = np.multiply(np.divide(1, (np.pi*(distance))), np.sin(0.1*np.pi*(n-mid_point)))
# And re-enable the warnings
np.seterr(**old_settings);

### Result of applying de L'Hôpital's rule

 As we need to evaluate the filter response at the mid point separately,
 this is computed here.  The window response at the mid-point is 1, so it
 is just the result of applying de L'Hôpital's rule

In [ ]:
h_ideal[int(mid_point)] = 0.1

### Window
Now apply the Hann window.  If a different stopband criterion is required, this window value can be changed.

In [ ]:
h = np.multiply(h_ideal,np.hanning(M))

### Now display the filter taps
Here the delay that is introduced by the filter is evident as the peak is located in the middle of the impulse response, and in particular, not at delay 0.

In [ ]:
plot_stem(n, h, M, 'FIR_311_tap_impulse_response.pdf')

### Compute and plot the frequency response using an fft
We then convert the result to the dB scale for plotting.

In [ ]:
FFT_length = 4096
f = np.multiply((1/FFT_length), np.arange(0, FFT_length))
H = 20*np.log10(abs(np.fft.fft(h,FFT_length)))

plot_freq(f, H, 'FIR_311_tap_frequency_response.pdf')

###  Repeat for an even length filter

In [ ]:
M = 310
mid_point = (M+1) / 2
n = np.arange(0, M)

distance = n - mid_point + 1

h = np.multiply(np.multiply(np.divide(1, (np.pi*(distance))), np.sin(0.1*np.pi*(n-mid_point+1))),
               np.hanning(M))

# Calculate the transfer function
fft = np.fft.fft(h,FFT_length)

# For displaying as a transfer function, it is useful to have a very small value,
# but not zero, before calculating the dB equivalent.  numpy.where allows all
# zero elements to be substituted by a non-zero value:
fft = np.where(fft == 0, 1e-99, fft)

# Convert to dBs
H = 20*np.log10(abs(fft))

Plot the figure

In [ ]:
plot_stem(n, h, M, 'FIR_310_tap_impulse_response.pdf')
plot_freq(f, H, 'FIR_310_tap_frequency_response.pdf')

© The University of Edinburgh: Produced by D. Laurenson, School of Engineering. Initial code conversion by Xing Zixiao.